In [24]:
# h5py 안 될 때
#!brew reinstall hdf5
#!export CPATH="/opt/homebrew/include/"
#!export HDF5_DIR=/opt/homebrew/
#!python3 -m pip install h5py

In [25]:
import pickle
import math
from copy import deepcopy

import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn

In [26]:
path_root = '../'
path_container = './Container/'

### Dataset 가져오기 (1) dict

In [27]:
with open(path_container + 'id_cuisine_dict.pickle', 'rb') as f:
    id_cuisine_dict = pickle.load(f)

print(id_cuisine_dict)
type(id_cuisine_dict), len(id_cuisine_dict)

{0: 'brazilian', 1: 'british', 2: 'cajun_creole', 3: 'chinese', 4: 'filipino', 5: 'french', 6: 'greek', 7: 'indian', 8: 'irish', 9: 'italian', 10: 'jamaican', 11: 'japanese', 12: 'korean', 13: 'mexican', 14: 'moroccan', 15: 'russian', 16: 'southern_us', 17: 'spanish', 18: 'thai', 19: 'vietnamese'}


(dict, 20)

In [28]:
with open(path_container + 'cuisine_id_dict.pickle', 'rb') as f:
    cuisine_id_dict = pickle.load(f)

print(cuisine_id_dict)
type(cuisine_id_dict), len(cuisine_id_dict)

{'brazilian': 0, 'british': 1, 'cajun_creole': 2, 'chinese': 3, 'filipino': 4, 'french': 5, 'greek': 6, 'indian': 7, 'irish': 8, 'italian': 9, 'jamaican': 10, 'japanese': 11, 'korean': 12, 'mexican': 13, 'moroccan': 14, 'russian': 15, 'southern_us': 16, 'spanish': 17, 'thai': 18, 'vietnamese': 19}


(dict, 20)

In [29]:
with open(path_container + 'id_ingredient_dict.pickle', 'rb') as f:
    id_ingredient_dict = pickle.load(f)

print(id_ingredient_dict[6694])
type(id_ingredient_dict), len(id_ingredient_dict)

Franks Hot Sauce


(dict, 6714)

In [30]:
with open(path_container + 'ingredient_id_dict.pickle', 'rb') as f:
    ingredient_id_dict = pickle.load(f)

print(ingredient_id_dict['lemon grass'])
print(ingredient_id_dict['lemongrass'])
print(ingredient_id_dict['bread'])
type(ingredient_id_dict), len(ingredient_id_dict)

37
561
[698, 1039, 4315]


(dict, 6694)

### Datastet 가져오기 (2) h5py

In [31]:
"""
_train = [arr_data_train, bin_data_train, int_labels_train, bin_labels_train]
_valid_class = [arr_data_valid_class, bin_data_valid_class, int_labels_valid_class, bin_labels_valid_class]
_valid_compl = [arr_data_valid_compl, bin_data_valid_compl, int_labels_valid_compl, bin_labels_valid_compl]
_test_class = [arr_data_test_class, bin_data_test_class, None, None]
_test_compl = [arr_data_test_compl, bin_data_test_compl, None, None]
"""

'\n_train = [arr_data_train, bin_data_train, int_labels_train, bin_labels_train]\n_valid_class = [arr_data_valid_class, bin_data_valid_class, int_labels_valid_class, bin_labels_valid_class]\n_valid_compl = [arr_data_valid_compl, bin_data_valid_compl, int_labels_valid_compl, bin_labels_valid_compl]\n_test_class = [arr_data_test_class, bin_data_test_class, None, None]\n_test_compl = [arr_data_test_compl, bin_data_test_compl, None, None]\n'

In [32]:
with h5py.File(path_container + 'train', 'r') as f:
    bin_data_train = f['bin_data'][:]
    int_labels_train = f['int_labels'][:]
    bin_labels_train = f['bin_labels'][:]

print(bin_data_train.shape)
print(int_labels_train.shape)
print(bin_labels_train.shape)

(23547, 6714)
(23547,)
(23547, 20)


In [33]:
with h5py.File(path_container + 'valid_class', 'r') as f:
    bin_data_valid_class = f['bin_data'][:]
    int_labels_valid_class = f['int_labels'][:]
    bin_labels_valid_class = f['bin_labels'][:]

print(bin_data_valid_class.shape)
print(int_labels_valid_class.shape)
print(bin_labels_valid_class.shape)

(7848, 6714)
(7848,)
(7848, 20)


In [34]:
with h5py.File(path_container + 'valid_compl', 'r') as f:
    bin_data_valid_compl = f['bin_data'][:]
    int_labels_valid_compl = f['int_labels'][:]
    bin_labels_valid_compl = f['bin_labels'][:]

print(bin_data_valid_compl.shape)
print(int_labels_valid_compl.shape)
print(bin_labels_valid_compl.shape)

(7848, 6714)
(7848,)
(7848, 6714)


In [35]:
with h5py.File(path_container + 'test_class', 'r') as f:
    bin_data_test_class = f['bin_data'][:]

print(bin_data_test_class.shape)

(3924, 6714)


In [36]:
with h5py.File(path_container + 'test_compl', 'r') as f:
    bin_data_test_compl = f['bin_data'][:]

print(bin_data_test_compl.shape)

(3924, 6714)


In [66]:
# binary to int
x = np.full((2, 6714), 6714) 
for i, bin_recipe in enumerate(bin_data_train[:2]):
    recipe = np.arange(6714)[bin_recipe==1]
    x[i][:len(recipe)] = recipe
x

array([[2813, 3146, 3229, ..., 6714, 6714, 6714],
       [ 392,  937, 1476, ..., 6714, 6714, 6714]])

### Ingredient feature vectors (nn.Embedding)

In [67]:
num_ingredients = pad_idx +1  # = len(id_ingredient_dict) + 1  # +1 for padding (last)
max_length = 65  # maximum num of ingredients per recipe
embedding_dim = 256


Embed = nn.Embedding(num_embeddings=num_ingredients, embedding_dim=embedding_dim, padding_idx=-1)
x = torch.LongTensor(arr_data_train[:2])

print(Embed.weight)
print(Embed.weight.size())

Parameter containing:
tensor([[ 3.2728,  2.0576,  1.7179,  ..., -0.2083, -0.1903,  0.6921],
        [-0.8021,  0.2044, -0.7084,  ...,  0.3418,  1.1189, -0.9387],
        [-1.4479, -0.9402,  0.0745,  ...,  1.0755, -0.6538,  1.6401],
        ...,
        [-0.9674, -1.1701, -0.5552,  ..., -0.1652,  0.0517,  0.1858],
        [-0.6495,  0.1527,  0.7819,  ..., -2.0525,  0.0562, -0.0381],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       requires_grad=True)
torch.Size([6715, 256])


In [68]:
with torch.no_grad():
    print(Embed.weight[0])

tensor([ 3.2728e+00,  2.0576e+00,  1.7179e+00, -2.7353e-01,  1.4686e-03,
         2.3524e-02, -8.1430e-01,  1.1356e+00,  4.7671e-01, -1.0477e+00,
         5.3869e-01, -1.0514e+00,  2.7868e+00, -3.4840e-01,  1.0300e+00,
        -8.8978e-01,  8.7044e-01,  6.7304e-01, -9.6107e-01, -1.5418e-01,
        -2.4102e-01,  1.8187e+00, -8.8296e-01,  1.0660e+00, -1.5905e+00,
        -2.2823e-01,  2.2334e-01, -3.6037e-01,  1.2920e-01, -2.1590e-01,
        -5.8105e-02,  8.6467e-02, -1.0668e+00,  1.3349e+00,  4.5772e-01,
        -2.2714e+00, -1.6354e+00, -1.4927e+00, -1.6907e+00, -7.7145e-01,
        -1.9655e+00, -4.6343e-01,  2.0147e+00,  1.2299e-01, -1.5589e+00,
        -2.7378e+00,  1.5156e+00, -9.3444e-01, -1.0782e-01, -1.4461e+00,
         1.4417e+00,  2.2120e+00, -4.1280e-01, -2.2266e+00,  1.9953e+00,
        -9.6570e-01,  1.3736e+00, -4.4300e-01, -6.0772e-02,  6.9036e-01,
        -4.0267e-01,  1.0846e+00,  1.8355e+00,  8.6360e-01, -8.8971e-01,
         2.0524e+00, -1.6007e+00,  1.5747e+00, -1.7

In [69]:
print(x)
with torch.no_grad():
    print(Embed(x).size())
    print(Embed(x)[0][11]) # zero padded feature matrix! (0번째 recipe는 9개라 10~65번째 feature vector는 zeros.)

tensor([[2813, 3146, 3229, 3885, 4379, 4390, 5250, 5456, 6187, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714],
        [ 392,  937, 1476, 2172, 2351, 2813, 3350, 3554, 3857, 3978, 5249, 5648,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714]])
torch.Size([2, 65, 256])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

### Models

Set transformer: ( https://github.com/juho-lee/set_transformer )

In [70]:
## Building blocks of Set Transformers ##
# added masks.

class MAB(nn.Module):
    def __init__(self, dim_Q, dim_K, dim_V, num_heads, ln=False, dropout=0.2):
        super(MAB, self).__init__()
        self.dim_V = dim_V
        self.num_heads = num_heads
        self.fc_q = nn.Linear(dim_Q, dim_V)
        self.fc_k = nn.Linear(dim_K, dim_V)
        self.fc_v = nn.Linear(dim_K, dim_V)
        if ln:
            self.ln0 = nn.LayerNorm(dim_V)
            self.ln1 = nn.LayerNorm(dim_V)
        self.fc_o = nn.Sequential(
            nn.Linear(dim_V, 2*dim_V),
            nn.ReLU(),
            nn.Linear(2*dim_V, dim_V))
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, Q, K, mask=None):
        # Q (batch, q_len, d_hid)
        # K (batch, k_len, d_hid)
        # V (batch, v_len, d_hid == dim_V)
        Q = self.fc_q(Q)
        K, V = self.fc_k(K), self.fc_v(K)
        
        dim_split = self.dim_V // self.num_heads
        
        # Q_ (batch * num_heads, q_len, d_hid // num_heads)
        # K_ (batch * num_heads, k_len, d_hid // num_heads)
        # V_ (batch * num_heads, v_len, d_hid // num_heads)
        Q_ = torch.cat(Q.split(dim_split, 2), 0)
        K_ = torch.cat(K.split(dim_split, 2), 0)
        V_ = torch.cat(V.split(dim_split, 2), 0)
        
        # energy (batch * num_heads, q_len, k_len)
        energy = Q_.bmm(K_.transpose(1,2))/math.sqrt(self.dim_V)
        if mask is not None:
            energy.masked_fill_(mask, float('-inf'))
        A = torch.softmax(energy, 2)
        
        # O (batch, q_len, d_hid)
        O = torch.cat((Q_ + A.bmm(V_)).split(Q.size(0), 0), 2)
        O = O if getattr(self, 'ln0', None) is None else self.ln0(O)
        O = O + self.dropout(self.fc_o(O))
        O = O if getattr(self, 'ln1', None) is None else self.ln1(O)
        return O

class SAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, ln=False, dropout=0.2):
        super(SAB, self).__init__()
        self.mab = MAB(dim_in, dim_in, dim_out, num_heads, ln=ln, dropout=dropout)

    def forward(self, X, mask=None):
        return self.mab(X, X, mask=mask)

class ISAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, num_inds, ln=False, dropout=0.2):
        super(ISAB, self).__init__()
        self.I = nn.Parameter(torch.Tensor(1, num_inds, dim_out))
        nn.init.xavier_uniform_(self.I)
        self.mab0 = MAB(dim_out, dim_in, dim_out, num_heads, ln=ln, dropout=dropout)
        self.mab1 = MAB(dim_in, dim_out, dim_out, num_heads, ln=ln, dropout=dropout)

    def forward(self, X, mask=None):
        H = self.mab0(self.I.repeat(X.size(0), 1, 1), X, mask=mask)
        return self.mab1(X, H)

class PMA(nn.Module):
    def __init__(self, dim, num_heads, num_seeds, ln=False, dropout=0.2):
        super(PMA, self).__init__()
        self.S = nn.Parameter(torch.Tensor(1, num_seeds, dim))
        nn.init.xavier_uniform_(self.S)
        self.mab = MAB(dim, dim, dim, num_heads, ln=ln, dropout=dropout)
        
    def forward(self, X, mask=None):
        return self.mab(self.S.repeat(X.size(0), 1, 1), X, mask=mask)

In [71]:
class Encoder(nn.Module):
    def __init__(self, dim_input,
                 num_inds=32, dim_hidden=128, num_heads=4, num_layers=2, ln=False):
        super(Encoder, self).__init__()
        self.enc = nn.ModuleList(
            [ISAB(dim_input, dim_hidden, num_heads, num_inds, ln=ln)] +
            [ISAB(dim_hidden, dim_hidden, num_heads, num_inds, ln=ln) for _ in range(num_layers-1)]
        )
        
    def forward(self, X, mask=None):
        out = X.detach()
        for module in self.enc:
            out = module(out, mask=mask)
        return out

class Decoder(nn.Module):
    def __init__(self, num_outputs,
                 num_inds=32, dim_hidden=128, num_heads=4, ln=False):
        super(Decoder, self).__init__()
        self.dec = nn.Sequential(
                PMA(dim_hidden, num_heads, num_outputs, ln=ln),
                SAB(dim_hidden, dim_hidden, num_heads, ln=ln),
                SAB(dim_hidden, dim_hidden, num_heads, ln=ln))
        
    def forward(self, X):
        return self.dec(X)

In [98]:
print(1 \
     +2)

3


#### Examples

In [72]:
x = torch.LongTensor(arr_data_train[:2])

with torch.no_grad():
    print('input size', x.size())
    print('input', x)
    feature = Embed(x)  # Size 2, 65, 256
    print('feature size', feature.size())
    print('feature', feature)

input size torch.Size([2, 65])
input tensor([[2813, 3146, 3229, 3885, 4379, 4390, 5250, 5456, 6187, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714],
        [ 392,  937, 1476, 2172, 2351, 2813, 3350, 3554, 3857, 3978, 5249, 5648,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714]])
feature size torch.Size([2, 65, 256])
feature tensor([[[-0.5348, -1.4913

In [97]:
x

tensor([[2813, 3146, 3229, 3885, 4379, 4390, 5250, 5456, 6187, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714],
        [ 392,  937, 1476, 2172, 2351, 2813, 3350, 3554, 3857, 3978, 5249, 5648,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714]])

In [74]:
x1 = deepcopy(x)
x1[0][:2] = x[0][[1,0]]
print('x1 size', x1.size())
print('x1', x1)
with torch.no_grad():
    feature1 = Embed(x1)
    print('feature1', feature1)

x1 size torch.Size([2, 65])
x1 tensor([[3146, 2813, 3229, 3885, 4379, 4390, 5250, 5456, 6187, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714],
        [ 392,  937, 1476, 2172, 2351, 2813, 3350, 3554, 3857, 3978, 5249, 5648,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714]])
feature1 tensor([[[ 0.8665,  1.1376, -0.3240,  ..., -2.0181,  1.4959, -1.2283]

In [75]:
_sab = SAB(256, 128, 4)

In [76]:
with torch.no_grad():
    z = _sab(feature, mask=enc_mask)
    print('sab(feature)',z)
    print(z.size())

sab(feature) tensor([[[-2.5072e-02, -4.3051e-01,  1.2263e-01,  ...,  1.0302e+00,
          -1.1187e+00, -6.9870e-01],
         [ 5.2234e-01, -3.1272e-01,  1.7651e-01,  ..., -1.1486e-01,
          -1.5416e-01,  6.6038e-01],
         [-1.1277e+00, -3.8550e-01, -1.7317e-01,  ...,  7.0273e-01,
           2.1384e-01, -1.5691e-01],
         ...,
         [-5.6566e-02, -1.0434e-01,  8.8718e-02,  ...,  1.7007e-01,
          -1.1468e-01, -1.5865e-01],
         [-5.6566e-02, -1.0434e-01,  8.8718e-02,  ...,  1.7007e-01,
           4.3089e-03, -1.5865e-01],
         [-3.2818e-03, -8.6689e-02,  8.8718e-02,  ...,  1.7913e-01,
          -1.1468e-01, -1.5865e-01]],

        [[ 2.0383e-01,  8.7391e-01,  9.2565e-02,  ...,  4.2228e-01,
          -7.6782e-01, -9.0063e-01],
         [ 5.9094e-01, -4.9040e-01,  3.3007e-01,  ...,  1.5430e+00,
          -1.3899e+00, -1.3009e-01],
         [ 1.5809e+00,  1.6993e+00,  7.7491e-01,  ...,  8.7152e-02,
          -2.0340e-01,  6.1534e-01],
         ...,
         [-5

In [77]:
_enc = Encoder(256, dim_hidden=128) 

In [78]:
with torch.no_grad():
    code = _enc(feature,mask=enc_mask)
    print(feature.size())
    print(code[0][15:])
    print(code.size())  # Size 2, 65, dim_hidden

torch.Size([2, 65, 256])
tensor([[-0.1059, -0.3514, -0.0556,  ..., -0.1083, -0.3708, -0.0339],
        [-0.0847, -0.3444, -0.0712,  ..., -0.1353, -0.3869, -0.0893],
        [ 0.0056, -0.3685, -0.0652,  ..., -0.1404, -0.3679, -0.1152],
        ...,
        [-0.1174, -0.3425, -0.0815,  ..., -0.1387, -0.3789, -0.0180],
        [-0.0978, -0.3619, -0.0729,  ..., -0.1695, -0.3428, -0.1099],
        [-0.1119, -0.1719, -0.0346,  ..., -0.1596, -0.3800, -0.1061]])
torch.Size([2, 65, 128])


In [79]:
with torch.no_grad():
    code1 = _enc(feature1, mask=enc_mask)
    print(code1)
    print(code1.size())  # Size 2, 65, dim_hidden  # permutation equivariant! (observe first 2 rows)

tensor([[[ 2.3806e-01, -1.6879e-01, -1.5313e-01,  ...,  4.2572e-02,
          -6.1262e-01, -1.3449e-01],
         [ 8.8945e-01, -5.3579e-02, -4.0432e-01,  ..., -5.3693e-01,
          -2.5833e-01,  2.4623e-01],
         [-2.6637e-01, -1.1682e-01,  4.5651e-01,  ..., -7.3169e-02,
          -3.3885e-01, -3.0409e-04],
         ...,
         [-1.0347e-01, -3.6286e-01, -5.0822e-02,  ..., -1.4331e-01,
          -3.7042e-01, -3.4987e-02],
         [-1.0872e-01, -3.9430e-01, -3.1552e-02,  ..., -1.6423e-01,
          -3.8089e-01, -4.6489e-02],
         [-7.7637e-03, -3.6729e-01,  2.1948e-02,  ..., -1.6029e-01,
          -3.8810e-01, -1.1545e-01]],

        [[-6.5920e-02, -6.1602e-02, -5.3671e-01,  ..., -9.9114e-02,
           5.2724e-01,  3.9178e-01],
         [ 3.8000e-01, -1.2891e-01,  9.4537e-01,  ...,  6.6126e-02,
           5.6672e-01,  3.8232e-01],
         [-3.8756e-03, -9.7366e-01,  4.4759e-01,  ...,  2.5574e-02,
           1.2526e-01,  3.3804e-01],
         ...,
         [-9.8407e-02, -2

In [80]:
_dec = Decoder(1, dim_hidden=128)

In [81]:
with torch.no_grad():
    out = _dec(code)
    print(out)
    print(out.size())  # Size 2, num_output, dim_hidden. FF 적용 직전 vector.

tensor([[[ 0.1497, -0.0521,  0.0675,  0.0652, -0.0708, -0.0593, -0.2947,
          -0.1471,  0.0851,  0.0566,  0.0065, -0.1735,  0.0025, -0.0995,
           0.3536,  0.1726,  0.0151, -0.0533,  0.3726, -0.1958,  0.3381,
           0.0911, -0.0272,  0.1099,  0.0124, -0.1490, -0.0267,  0.2880,
          -0.1414,  0.1710, -0.1190,  0.0281, -0.0124,  0.1056,  0.1193,
          -0.1263, -0.0005, -0.0351, -0.1742,  0.0507,  0.0696, -0.0749,
          -0.1624, -0.3835, -0.0924,  0.0014,  0.1815,  0.1520,  0.0976,
          -0.1666, -0.2210,  0.0397, -0.1481, -0.0580, -0.0651,  0.0135,
           0.1413,  0.2000,  0.0184, -0.0768,  0.0059,  0.0186, -0.0755,
           0.0342,  0.0811, -0.1843, -0.1589,  0.0689,  0.0355, -0.1432,
           0.0241, -0.0497, -0.1559,  0.2053,  0.1894,  0.0601,  0.0463,
          -0.1679, -0.0293,  0.0903, -0.1158,  0.0468,  0.1738,  0.0413,
          -0.2796,  0.0191, -0.3501,  0.1799,  0.0060, -0.0295,  0.0369,
          -0.0476, -0.0276,  0.0278,  0.0089,  0.09

In [82]:
# permutation invariance

with torch.no_grad():
    x2 = deepcopy(x)
    """3146, 2813, 3229, 3885, 4379, 4390, 5250, 5456, 6187"""
    shuf_ind = torch.randperm(x2.size(1))
    x2[0] = x2[0][shuf_ind]  # ramdom permuted!
    print(x2[0])
    feature2 = Embed(x2)
    enc_mask2 = deepcopy(enc_mask)
    enc_mask2[0] = enc_mask[0][0][shuf_ind]
    code2 = _enc(feature2, mask=enc_mask2)
    out2 = _dec(code2)
    print(out2.size())
    print(out2[0].isclose(out[0]))
    print(out2[0] - out[0]) # almost the same.

tensor([4379, 6714, 4390, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
        6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 3146, 6714,
        6714, 2813, 6714, 6714, 6714, 6714, 6714, 6714, 6187, 6714, 6714, 6714,
        6714, 6714, 6714, 6714, 6714, 5250, 5456, 6714, 6714, 6714, 6714, 6714,
        6714, 3229, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
        3885, 6714, 6714, 6714, 6714])
torch.Size([2, 1, 128])
tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False

### Unified Model: Classification + Completion (CCNet)

In [83]:
x

tensor([[2813, 3146, 3229, 3885, 4379, 4390, 5250, 5456, 6187, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714],
        [ 392,  937, 1476, 2172, 2351, 2813, 3350, 3554, 3857, 3978, 5249, 5648,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714]])

In [84]:
def make_one_hot(x):
    if type(x) is not torch.Tensor:
        x = torch.LongTensor(x)
    if x.dim() > 2:
        x = x.squeeze()
        if x.dim() > 2:
            return False
    elif x.dim() < 2:
        x = x.unsqueeze(0)
    return F.one_hot(x).sum(1)[:,:-1]

In [85]:
make_one_hot(x).size()

torch.Size([2, 6714])

In [90]:
class CCNet(nn.Module):
    def __init__(self, dim_input=256,
                 dim_output=20,
                 num_items=6714+1, 
                 num_inds=32, 
                 dim_hidden=128, 
                 num_heads=4, 
                 num_outputs=1+1,  # classification 1 + completion 1
                 num_enc_layers=4, 
                 num_dec_layers=2,
                 ln=True,          # LayerNorm option
                 dropout=0.2,      # Dropout option
                 classify=True,    # completion만 하고 싶으면 False로
                 complete=True):   # classification만 하고 싶으면 False로
        super(CCNet, self).__init__()
        
        self.num_heads = num_heads
        self.padding_idx = num_items-1
        self.classify, self.complete = classify, complete
        self.embedding =  nn.Embedding(num_embeddings=num_items, embedding_dim=dim_input, padding_idx=-1)
        self.encoder = nn.ModuleList(
            [ISAB(dim_input, dim_hidden, num_heads, num_inds, ln=ln)] +
            [ISAB(dim_hidden, dim_hidden, num_heads, num_inds, ln=ln) for _ in range(num_enc_layers-1)])
        self.pooling = PMA(dim_hidden, num_heads, num_outputs, ln=ln)
        self.decoder1 = nn.Sequential(
            *[SAB(dim_hidden, dim_hidden, num_heads, ln=ln, dropout=dropout) for _ in range(num_dec_layers)])
        self.ff1 = nn.Sequential(
            nn.Linear(dim_hidden, dim_hidden),
            nn.ReLU(),
            nn.Linear(dim_hidden, dim_output))
        self.decoder2 = nn.ModuleList(
            [MAB(dim_hidden, dim_input, dim_hidden, num_heads, ln=ln, dropout=dropout) for _ in range(2)])
        self.ff2 = nn.Linear(dim_hidden, num_items-1)
    
    def make_one_hot(x):
        """ Convert int_data into bin_data, if needed. """
        if type(x) is not torch.Tensor:
            x = torch.LongTensor(x)
        if x.dim() > 2:
            x = x.squeeze()
            if x.dim() > 2:
                return False
        elif x.dim() < 2:
            x = x.unsqueeze(0)
        return F.one_hot(x).sum(1)[:,:-1]
    
    def forward(self, x): 
        # x(=recipes): (batch, max_num_ingredient=65) : int_data.
        if not (self.classify or self.complete):
            return
        
        x = torch.LongTensor(x)
        feature = self.embedding(x)
        # feature: (batch, max_num_ingredient=65, dim_input=256)
        # cf. embedding.weight: (num_items=6715, dim_input=256)
        mask = (x == self.padding_idx).repeat(self.num_heads,1).unsqueeze(1)
        # mask: (batch*num_heads, 1, max_num_ingredient=65)
        code = feature.clone()
        for module in self.encoder:
            code = module(code, mask=mask)
        # code: (batch, max_num_ingredient=65, dim_hidden=128) : permutation-equivariant.
        
        pooled = self.pooling(code, mask=mask)
        # pooled: (batch, num_outputs=2, dim_hidden=128) : permutation-invariant.
        
        signals = self.decoder1(pooled)
        # no mask; signals: (batch, num_outputs=2, dim_hidden=128) : permutation-invariant.
        
        # split two signals: for classification & completion.
        signal_classification = signals[:][0]                   # (batch, dim_hidden=128)
        signal_completion = signals.clone()[:][1].unsqueeze(1)  # (batch, 1, dim_hidden=128)
        
        # Classification:
        if self.classify:
            logit_classification = self.ff1(signal_classification)  # (batch, dim_output)
            if not self.complete:
                return logit_clasification
        
        # Completion:
        if self.complete:
            bool_x = (make_one_hot(x) == True)
            used_ingred_mask = bool_x.repeat(self.num_heads,1).unsqueeze(1)
            # used_ingred_mask: (batch*num_heads, 1, num_items-1=6714)
            
            embedding_weight = self.embedding.weight[:-1].unsqueeze(0).repeat(feature.size(0),1,1)
            # embedding_weight: (batch, num_items=6715, dim_input=256)
            
            for module in self.decoder2:
                signal_completion = module(signal_completion, embedding_weight, mask=used_ingred_mask)
            logit_completion = self.ff2(signal_completion.squeeze()) # (batch, num_items-1=6714)
            logit_completion[bool_x] = float('-inf')
            if not self.classify:
                return logit_completion

        return logit_classification, logit_completion

In [91]:
_model = CCNet()

In [92]:
with torch.no_grad():
    logit1, logit2 = _model(arr_data_train[:2])
    print(logit1, logit1.size())
    print(logit2, logit2.size())
    

tensor([[ 0.0134,  0.0076,  0.3617, -0.0838,  0.1405, -0.3312, -0.2302, -0.2888,
          0.2556, -0.1658,  0.1963, -0.1480, -0.2014,  0.5930, -0.0235,  0.1583,
         -0.2339, -0.4005,  0.1152, -0.3462],
        [ 0.0284, -0.0381,  0.4265, -0.0877,  0.1785, -0.3441, -0.2454, -0.3424,
          0.2262, -0.2049,  0.2100, -0.1222, -0.1992,  0.6614,  0.0021,  0.1020,
         -0.2523, -0.2883,  0.0336, -0.3439]]) torch.Size([2, 20])
tensor([[-0.3969, -0.0151,  0.0779,  ..., -0.7825,  0.2570, -1.4711],
        [-0.3927,  0.1244, -0.0351,  ..., -0.9459,  0.2584, -1.6961]]) torch.Size([2, 6714])


In [93]:
# Sanity check
x = torch.LongTensor(arr_data_train[:2])
bin_x = make_one_hot(x)

print((F.softmax(logit2, dim=-1) == 0).any())
logit2[0][torch.where(bin_x[0]==1)[0]]

tensor(True)


tensor([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf])